## Data Cleaning
Code written by Anna Haas and edited by Julia Taussig

In [1]:
#Imports:

import numpy as np                # import numpy
import pandas as pd               # import pandas

import matplotlib.pyplot as plt   # import matplotlib
import seaborn as sns             # import seaborn

%matplotlib inline

In [3]:
# Read in train and test
train = pd.read_csv('./data/large_train_sample.csv')
test = pd.read_csv('./data/test_data.csv')

In [5]:
train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country,wage
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Female,0,0,40,Cuba,<=50K


In [6]:
train['sex'].value_counts()

 Male      21790
 Female    10771
Name: sex, dtype: int64

In [7]:
# Fix 'sex' column - Train (1 if male, 0 if female)
train['sex'] = train['sex'].map(lambda x: x.lower().strip())
train['sex'] = train['sex'].map(lambda x: 1 if  x == 'male' else 0)

In [8]:
train['sex'].value_counts()

1    21790
0    10771
Name: sex, dtype: int64

In [9]:
test['sex'].value_counts()

 Male      10860
 Female     5421
Name: sex, dtype: int64

In [10]:
# Fix 'sex' column - TEST
test['sex'] = test['sex'].map(lambda x: x.lower().strip())
test['sex'] = test['sex'].map(lambda x: 1 if  x == 'male' else 0)

In [11]:
test['sex'].value_counts()

1    10860
0     5421
Name: sex, dtype: int64

In [12]:
train['wage'].value_counts()

 <=50K    24720
 >50K      7841
Name: wage, dtype: int64

In [13]:
train['wage'].value_counts().index

Index([' <=50K', ' >50K'], dtype='object')

In [14]:
train['wage'] = train['wage'].map(lambda x: 0 if x.strip() == '<=50K' else 1)

In [15]:
# 1 if greater than 50,000
train['wage'].value_counts()

0    24720
1     7841
Name: wage, dtype: int64

In [16]:
# is_USA
train['is_USA'] = train['native-country'].map(lambda x: 1 if x.strip() == 'United-States' else 0)

In [17]:
train['relationship'] = train['relationship'].map(lambda x: x.strip())

In [18]:
train['relationship'].value_counts()

Husband           13193
Not-in-family      8305
Own-child          5068
Unmarried          3446
Wife               1568
Other-relative      981
Name: relationship, dtype: int64

In [19]:
train[train['relationship'] == 'Own-child']['wage'].value_counts(normalize=True)

0    0.98678
1    0.01322
Name: wage, dtype: float64

In [20]:
#train['in_family'] = train['relationship'].map(lambda x: 1 if x.strip() in ['Wife', 'Husband', 'Own-child'] else 0)

In [21]:
# Create dummies for relationship columns
train['is_married'] = train['marital-status'].map(lambda x: 1 if x.strip() in ['Married-civ-spouse', 'Married-AF-spouse'] else 0)

In [22]:
train[['is_married', 'wage']].corr()

,is_married,wage
is_married,1.000000,0.445289
wage,0.445289,1.000000


In [23]:
train['is_child'] = train['relationship'].map(lambda x: 1 if x.strip() == 'Own-child' else 0)

In [24]:
train['is_single'] = train['relationship'].map(lambda x: 1 if x.strip() in ['Not-in-family', 'Unmarried'] else 0)
# Other-relative not included for colinearity reasons

In [25]:
#train.drop(columns=['relationship'], inplace=True)

In [26]:
train.corr()[abs(train.corr()['wage']) > .2][['wage']]

,wage
age,0.234037
education-num,0.335154
sex,0.215980
capital-gain,0.223329
hours-per-week,0.229689
wage,1.000000
is_married,0.445289
is_child,-0.228532
is_single,-0.262590


In [27]:
train_dum = pd.get_dummies(train, drop_first=True)

In [28]:
train_dum.corr()[abs(train_dum.corr()['wage']) > .1][['wage']].sort_values(by = 'wage')

,wage
marital-status_ Never-married,-0.318440
is_single,-0.262590
relationship_Own-child,-0.228532
is_child,-0.228532
relationship_Not-in-family,-0.188497
occupation_ Other-service,-0.156348
relationship_Unmarried,-0.142857
education_ HS-grad,-0.131189
relationship_Wife,0.123264
education_ Doctorate,0.132572


In [29]:
from itertools import combinations

In [30]:
# See what numerical features would be best as interaction terms

# Remove all categorical columns from train and set to new dataframe
num_train = train_dum[train_dum._get_numeric_data().columns]

# Remove target
num_train.drop(columns = 'wage', inplace=True)

# Iterate through random combinations of 2 features to find the ones that create the highest correlated 
# interaction feature with wage
combos = list(combinations(num_train, 2))

best_interaction_terms = []

for predictor_set in combos:

    predictor_list = list(predictor_set)
    current_corr = np.corrcoef(train_dum[predictor_list[0]] * train_dum[predictor_list[1]], train_dum['wage'])[0][1]

    if abs(current_corr) > 0.45:
        print('Best Terms: ', predictor_set)
        print('Best Correlation: ', current_corr)
        print()

Best Terms:  ('education-num', 'is_married')
Best Correlation:  0.5253091020653882

Best Terms:  ('education-num', 'marital-status_ Married-civ-spouse')
Best Correlation:  0.5246618564678448



/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2530: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2531: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Best Terms:  ('hours-per-week', 'is_married')
Best Correlation:  0.4591101706819833

Best Terms:  ('hours-per-week', 'marital-status_ Married-civ-spouse')
Best Correlation:  0.4586187543382134



In [31]:
# Compare interaction to original correlations
print('Education-num: ', np.corrcoef(train_dum['education-num'], train_dum['wage'])[0][1])
print('Is_married: ', np.corrcoef(train_dum['is_married'], train_dum['wage'])[0][1])
print('Interaction Term: ', np.corrcoef(train_dum['education-num'] * train_dum['is_married'], train_dum['wage'])[0][1])

Education-num:  0.3351539526909413
Is_married:  0.44528883971607686
Interaction Term:  0.5253091020653882


In [32]:
train_dum['interaction'] = train_dum['is_married'] *train_dum['education-num'] 

In [33]:
from sklearn.linear_model import LogisticRegression, LassoCV, RidgeCV
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score, mean_squared_error

In [34]:
X = train_dum[['interaction']]
y = train_dum['wage']

In [35]:
# Train/test split (90/10 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=42)

# Scaling
# ss = StandardScaler()
# ss.fit(X_train)
# X_train_s = ss.transform(X_train)
# X_test_s = ss.transform(X_test)

In [36]:
logreg = LogisticRegression()

logreg.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [37]:
# Score the model
print('Train Score: ', logreg.score(X_train, y_train))
print('Test Score: ', logreg.score(X_test, y_test))
print('CV Score: ', cross_val_score(logreg, X_train, y_train, cv=5).mean())

Train Score:  0.8188301938301938
Test Score:  0.8222290451335585
CV Score:  0.818830220937498


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [38]:
train[train['is_married'] == 1]['marital-status'].value_counts()

 Married-civ-spouse    14976
 Married-AF-spouse        23
Name: marital-status, dtype: int64

In [39]:
train[train['is_child'] == 1]['marital-status'].value_counts()

 Never-married            4485
 Divorced                  328
 Separated                  99
 Married-civ-spouse         95
 Married-spouse-absent      45
 Widowed                    15
 Married-AF-spouse           1
Name: marital-status, dtype: int64

In [40]:
train['never_married'] = train['marital-status'].map(lambda x: 1 if x.strip() == 'Never-married' else 0)
train['is_exec'] = train['occupation'].map(lambda x: 1 if x.strip() == 'Exec-managerial' else 0)

In [41]:
train.corr()['wage']

age               0.234037
fnlwgt           -0.009463
education-num     0.335154
sex               0.215980
capital-gain      0.223329
capital-loss      0.150526
hours-per-week    0.229689
wage              1.000000
is_USA            0.034470
is_married        0.445289
is_child         -0.228532
is_single        -0.262590
never_married    -0.318440
is_exec           0.214861
Name: wage, dtype: float64

In [42]:
# never_married, **is_married, hours-per-week, capital-loss, capital-gain, sex, **education-num, age
# train_dum['interaction'] = train_dum['is_married'] *train_dum['education-num'] 

train['is_married * education-num'] = train['is_married'] * train['education-num']

In [43]:
features = ['is_married * education-num', 'never_married', 'hours-per-week', 'capital-loss', 'capital-gain', 'sex', 
            'age', 'is_exec']

In [44]:
X = train[features]
y = train['wage']

In [45]:
# Train/test split (90/10 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)

# Scaling
ss = StandardScaler()
ss.fit(X_train)
X_train_s = ss.transform(X_train)
X_test_s = ss.transform(X_test)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  import sys
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  


In [46]:
logreg = LogisticRegression(solver='lbfgs', max_iter = 500)

logreg.fit(X_train_s, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [47]:
# Score the model
print('Train Score: ', logreg.score(X_train_s, y_train))
print('Test Score: ', logreg.score(X_test_s, y_test))
print('CV Score: ', cross_val_score(logreg, X_train, y_train, cv=5).mean())

Train Score:  0.8397624897624898
Test Score:  0.8458420341481391
CV Score:  0.8395986868725156


In [48]:
train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,...,hours-per-week,native-country,wage,is_USA,is_married,is_child,is_single,never_married,is_exec,is_married * education-num
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,1,2174,...,40,United-States,0,1,0,0,1,1,0,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,1,0,...,13,United-States,0,1,1,0,0,0,1,13
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,1,0,...,40,United-States,0,1,0,0,1,0,0,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,1,0,...,40,United-States,0,1,1,0,0,0,0,7
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,0,0,...,40,Cuba,0,0,1,0,0,0,0,13


In [592]:
pd.to_pickle(train, './data/train_clean.pkl')

In [49]:
train.groupby(by = 'education').mean()[['wage']].sort_values(by='wage')

,wage
education,
Preschool,0.000000
1st-4th,0.035714
5th-6th,0.048048
11th,0.051064
9th,0.052529
7th-8th,0.061920
10th,0.066452
12th,0.076212
HS-grad,0.159509


In [50]:
train['education']

0            Bachelors
1            Bachelors
2              HS-grad
3                 11th
4            Bachelors
5              Masters
6                  9th
7              HS-grad
8              Masters
9            Bachelors
10        Some-college
11           Bachelors
12           Bachelors
13          Assoc-acdm
14           Assoc-voc
15             7th-8th
16             HS-grad
17             HS-grad
18                11th
19             Masters
20           Doctorate
21             HS-grad
22                 9th
23                11th
24             HS-grad
25           Bachelors
26             HS-grad
27        Some-college
28             HS-grad
29             HS-grad
             ...      
32531        Bachelors
32532        Doctorate
32533        Bachelors
32534     Some-college
32535             12th
32536        Bachelors
32537          HS-grad
32538        Bachelors
32539        Doctorate
32540          HS-grad
32541          HS-grad
32542          HS-grad
32543      

In [51]:
# if preschool = wage is 0

education_map = {'Preschool': 0,
                '1st-4th': 1,
                '5th-6th': 2,
                '7th-8th': 3,
                '9th': 4,
                '10th': 4,
                '11th': 4,
                '12th': 5,
                'HS-grad': 6,
                'Some-college': 7,
                'Assoc-acdm': 8,
                'Assoc-voc': 8,
                'Bachelors': 9,
                'Masters': 10,
                'Prof-school': 11,
                'Doctorate': 12}

In [52]:
train['education_scale'] = train['education'].map(lambda x: x.strip())
train['education_scale'] = train['education_scale'].map(education_map)

In [53]:
train['education_scale'].value_counts()

6     10501
7      7291
9      5355
4      2622
8      2449
10     1723
3       646
11      576
5       433
12      413
2       333
1       168
0        51
Name: education_scale, dtype: int64

In [54]:
train.corr()['wage'].sort_values()

never_married                -0.318440
is_single                    -0.262590
is_child                     -0.228532
fnlwgt                       -0.009463
is_USA                        0.034470
capital-loss                  0.150526
is_exec                       0.214861
sex                           0.215980
capital-gain                  0.223329
hours-per-week                0.229689
age                           0.234037
education-num                 0.335154
education_scale               0.342418
is_married                    0.445289
is_married * education-num    0.525309
wage                          1.000000
Name: wage, dtype: float64

In [55]:
# Compare interaction to original correlations
print('Education-num: ', np.corrcoef(train['education-num'], train['wage'])[0][1])
print('Is_married: ', np.corrcoef(train['is_married'], train['wage'])[0][1])
print('Interaction Term: ', np.corrcoef(train['education-num'] * train['is_married'], train_dum['wage'])[0][1])
print('Interaction Term: ', np.corrcoef(train['education_scale'] * train['is_married'], train_dum['wage'])[0][1])

Education-num:  0.3351539526909413
Is_married:  0.44528883971607686
Interaction Term:  0.5253091020653882
Interaction Term:  0.5300512649681725


In [105]:
np.corrcoef(train['education_scale'], train_dum['wage'])

array([[1.        , 0.34241757],
       [0.34241757, 1.        ]])

In [56]:
train.groupby(by = 'workclass').mean()[['wage']].sort_values(by='wage')

,wage
workclass,
Never-worked,0.000000
Without-pay,0.000000
?,0.104031
Private,0.218673
State-gov,0.271957
Self-emp-not-inc,0.284927
Local-gov,0.294792
Federal-gov,0.386458
Self-emp-inc,0.557348


In [57]:
train.shape

(32561, 22)

In [58]:
train[train['workclass'] == ' ?'].shape

(1836, 22)

In [59]:
(1836 / 32561) * 100

5.638647461687294

In [60]:
train['unemployed'] = train['workclass'].map(lambda x: 1 if x.strip() in ['Never-worked', 'Without-pay', '?'] else 0)

In [61]:
train['gov_job'] = train['workclass'].map(lambda x: 1 if 'gov' in x.strip() else 0)

In [62]:
train['self_employed'] = train['workclass'].map(lambda x: 1 if 'Self-emp-inc' in x.strip() else 0)

In [63]:
train.corr()['wage']

age                           0.234037
fnlwgt                       -0.009463
education-num                 0.335154
sex                           0.215980
capital-gain                  0.223329
capital-loss                  0.150526
hours-per-week                0.229689
wage                          1.000000
is_USA                        0.034470
is_married                    0.445289
is_child                     -0.228532
is_single                    -0.262590
never_married                -0.318440
is_exec                       0.214861
is_married * education-num    0.525309
education_scale               0.342418
unemployed                   -0.079348
gov_job                       0.061903
self_employed                 0.139467
Name: wage, dtype: float64

In [64]:
train.groupby(by = 'occupation').mean()[['wage']].sort_values(by='wage')

,wage
occupation,
Priv-house-serv,0.006711
Other-service,0.041578
Handlers-cleaners,0.062774
?,0.103635
Armed-Forces,0.111111
Farming-fishing,0.115694
Machine-op-inspct,0.124875
Adm-clerical,0.134483
Transport-moving,0.200376


In [65]:
train.groupby(by = 'occupation').mean()[['wage']].sort_values(by='wage').index

Index([' Priv-house-serv', ' Other-service', ' Handlers-cleaners', ' ?',
       ' Armed-Forces', ' Farming-fishing', ' Machine-op-inspct',
       ' Adm-clerical', ' Transport-moving', ' Craft-repair', ' Sales',
       ' Tech-support', ' Protective-serv', ' Prof-specialty',
       ' Exec-managerial'],
      dtype='object', name='occupation')

In [66]:
train['top_job'] = train['occupation'].map(lambda x: 1 if x.strip() in ['Exec-managerial', 
                                                                        'Prof-specialty', 
                                                                        'Protective-serv', 
                                                                        'Tech-support'] else 0)

In [67]:
job_map = {' Priv-house-serv': 1, 
           ' Other-service': 1, 
           ' Handlers-cleaners': 1, 
           ' ?': 1,
           ' Armed-Forces': 2, 
           ' Farming-fishing': 2, 
           ' Machine-op-inspct': 2,
           ' Adm-clerical': 2, 
           ' Transport-moving': 3, 
           ' Craft-repair': 3, 
           ' Sales': 3,
           ' Tech-support': 4, 
           ' Protective-serv': 4, 
           ' Prof-specialty': 4,
           ' Exec-managerial': 5}

In [69]:
train['job_scale'] = train['occupation'].map(job_map)

In [70]:
abs(train.corr()['wage']).sort_values()

fnlwgt                        0.009463
is_USA                        0.034470
gov_job                       0.061903
unemployed                    0.079348
self_employed                 0.139467
capital-loss                  0.150526
is_exec                       0.214861
sex                           0.215980
capital-gain                  0.223329
is_child                      0.228532
hours-per-week                0.229689
age                           0.234037
is_single                     0.262590
top_job                       0.307887
never_married                 0.318440
education-num                 0.335154
job_scale                     0.339006
education_scale               0.342418
is_married                    0.445289
is_married * education-num    0.525309
wage                          1.000000
Name: wage, dtype: float64

In [71]:
# See what numerical features would be best as interaction terms

# Remove all categorical columns from train and set to new dataframe
num_train = train[train._get_numeric_data().columns]

# Remove target
num_train.drop(columns = ['wage', 'is_married * education-num'], inplace=True)

# Iterate through random combinations of 2 features to find the ones that create the highest correlated 
# interaction feature with wage
combos = list(combinations(num_train, 2))

best_interaction_terms = []

for predictor_set in combos:

    predictor_list = list(predictor_set)
    current_corr = np.corrcoef(train[predictor_list[0]] * train[predictor_list[1]], train['wage'])[0][1]

    if abs(current_corr) > 0.50:
        print('Best Terms: ', predictor_set)
        print('Best Correlation: ', current_corr)
        print()

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2530: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2531: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Best Terms:  ('education-num', 'is_married')
Best Correlation:  0.5253091020653882

Best Terms:  ('is_married', 'education_scale')
Best Correlation:  0.5300512649681725

Best Terms:  ('is_married', 'job_scale')
Best Correlation:  0.5185322685495046



In [72]:
train.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'sex', 'capital-gain',
       'capital-loss', 'hours-per-week', 'native-country', 'wage', 'is_USA',
       'is_married', 'is_child', 'is_single', 'never_married', 'is_exec',
       'is_married * education-num', 'education_scale', 'unemployed',
       'gov_job', 'self_employed', 'top_job', 'job_scale'],
      dtype='object')

In [73]:
train.drop(columns='is_married * education-num', inplace=True)

In [617]:
pd.to_pickle(train, './data/train_clean.pkl')

In [74]:
train[['marital-status', 'relationship', 'is_married']]

,marital-status,relationship,is_married
0,Never-married,Not-in-family,0
1,Married-civ-spouse,Husband,1
2,Divorced,Not-in-family,0
3,Married-civ-spouse,Husband,1
4,Married-civ-spouse,Wife,1
5,Married-civ-spouse,Wife,1
6,Married-spouse-absent,Not-in-family,0
7,Married-civ-spouse,Husband,1
8,Never-married,Not-in-family,0
9,Married-civ-spouse,Husband,1


In [75]:
train[train['relationship'] == 'Unmarried'][['marital-status', 'wage']].groupby(by='marital-status').mean()

,wage
marital-status,
Divorced,0.083698
Married-spouse-absent,0.038462
Never-married,0.026107
Separated,0.042129
Widowed,0.096606


In [76]:
train[train['relationship'] == 'Unmarried']['marital-status'].value_counts(normalize=True)

 Divorced                 0.464597
 Never-married            0.255659
 Separated                0.130876
 Widowed                  0.111143
 Married-spouse-absent    0.037725
Name: marital-status, dtype: float64

In [77]:
train[train['marital-status'] == ' Never-married'][['relationship', 'wage']].groupby(by='relationship').mean()

,wage
relationship,
Not-in-family,0.090523
Other-relative,0.013093
Own-child,0.007581
Unmarried,0.026107


In [78]:
train[train['marital-status'] == ' Never-married']['wage'].mean()

0.04596087241411589

In [79]:
train[train['relationship'] == 'Unmarried']['wage'].mean()

0.0632617527568195

In [80]:
train['was_married'] = train['marital-status'].map(lambda x: 1 if x.strip() in ['Divorced', 'Widowed', 'Separated'] else 0)

In [81]:
train[['is_married', 'was_married', 'never_married']].head()

,is_married,was_married,never_married
0,0,0,1
1,1,0,0
2,0,1,0
3,1,0,0
4,1,0,0


In [82]:
test = train['is_married'] +  train['was_married'] + train['never_married']

In [83]:
test.value_counts()

1    32143
0      418
dtype: int64

In [84]:
train['relationship'].value_counts()

Husband           13193
Not-in-family      8305
Own-child          5068
Unmarried          3446
Wife               1568
Other-relative      981
Name: relationship, dtype: int64

In [85]:
train['marital-status'].value_counts()

 Married-civ-spouse       14976
 Never-married            10683
 Divorced                  4443
 Separated                 1025
 Widowed                    993
 Married-spouse-absent      418
 Married-AF-spouse           23
Name: marital-status, dtype: int64

In [86]:
train[['is_married', 'was_married', 'never_married', 'wage']].corr()['wage'].sort_values()

never_married   -0.318440
was_married     -0.169632
is_married       0.445289
wage             1.000000
Name: wage, dtype: float64

In [87]:
train.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'sex', 'capital-gain',
       'capital-loss', 'hours-per-week', 'native-country', 'wage', 'is_USA',
       'is_married', 'is_child', 'is_single', 'never_married', 'is_exec',
       'education_scale', 'unemployed', 'gov_job', 'self_employed', 'top_job',
       'job_scale', 'was_married'],
      dtype='object')

In [88]:
test = train[['is_married', 'job_scale', 'education_scale', 
              'age', 'sex', 'hours-per-week', 'unemployed', 'gov_job', 'self_employed', 'capital-gain', 
              'capital-loss', 'wage']]

In [633]:
pd.to_pickle(test, './data/test_features.pkl')

In [89]:
train[train['is_child'] == 1]['wage'].value_counts(normalize=True)

0    0.98678
1    0.01322
Name: wage, dtype: float64

In [90]:
train_drop = train[train['occupation'] != ' ?']

In [91]:
train_drop.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'sex', 'capital-gain',
       'capital-loss', 'hours-per-week', 'native-country', 'wage', 'is_USA',
       'is_married', 'is_child', 'is_single', 'never_married', 'is_exec',
       'education_scale', 'unemployed', 'gov_job', 'self_employed', 'top_job',
       'job_scale', 'was_married'],
      dtype='object')

In [92]:
test_drop = train_drop[['is_married', 'job_scale', 'education_scale', 'age', 'sex', 'hours-per-week', 'unemployed', 'gov_job', 'self_employed', 'capital-gain', 'capital-loss', 'wage']]

In [640]:
pd.to_pickle(test, './data/test_drop_features.pkl')

In [93]:
new_train = train.drop(columns = ['fnlwgt', 'education-num', 'marital-status', 'relationship'])

In [94]:
new_train.drop(columns = ['education', 'never_married', 'is_exec', 'gov_job', 'self_employed', 'unemployed'], inplace=True)

In [95]:
new_train.drop(columns = ['is_child', 'is_single', 'top_job', 'was_married', 'native-country'])

,age,workclass,occupation,sex,capital-gain,capital-loss,hours-per-week,wage,is_USA,is_married,education_scale,job_scale
0,39,State-gov,Adm-clerical,1,2174,0,40,0,1,0,9,2
1,50,Self-emp-not-inc,Exec-managerial,1,0,0,13,0,1,1,9,5
2,38,Private,Handlers-cleaners,1,0,0,40,0,1,0,6,1
3,53,Private,Handlers-cleaners,1,0,0,40,0,1,1,4,1
4,28,Private,Prof-specialty,0,0,0,40,0,0,1,9,4
5,37,Private,Exec-managerial,0,0,0,40,0,1,1,10,5
6,49,Private,Other-service,0,0,0,16,0,0,0,4,1
7,52,Self-emp-not-inc,Exec-managerial,1,0,0,45,1,1,1,6,5
8,31,Private,Prof-specialty,0,14084,0,50,1,1,0,10,4
9,42,Private,Exec-managerial,1,5178,0,40,1,1,1,9,5


In [96]:
new_train['edu_scale * is_married'] = new_train['education_scale'] * new_train['is_married']

In [97]:
new_train['capital-net'] = new_train['capital-gain'] - new_train['capital-loss']

In [98]:
new_train.head()

,age,workclass,occupation,sex,capital-gain,capital-loss,hours-per-week,native-country,wage,is_USA,is_married,is_child,is_single,education_scale,top_job,job_scale,was_married,edu_scale * is_married,capital-net
0,39,State-gov,Adm-clerical,1,2174,0,40,United-States,0,1,0,0,1,9,0,2,0,0,2174
1,50,Self-emp-not-inc,Exec-managerial,1,0,0,13,United-States,0,1,1,0,0,9,1,5,0,9,0
2,38,Private,Handlers-cleaners,1,0,0,40,United-States,0,1,0,0,1,6,0,1,1,0,0
3,53,Private,Handlers-cleaners,1,0,0,40,United-States,0,1,1,0,0,4,0,1,0,4,0
4,28,Private,Prof-specialty,0,0,0,40,Cuba,0,0,1,0,0,9,1,4,0,9,0


In [99]:
new_train.drop(columns=['top_job', 'is_single', 'is_child'],inplace=True)

In [100]:
new_train

,age,workclass,occupation,sex,capital-gain,capital-loss,hours-per-week,native-country,wage,is_USA,is_married,education_scale,job_scale,was_married,edu_scale * is_married,capital-net
0,39,State-gov,Adm-clerical,1,2174,0,40,United-States,0,1,0,9,2,0,0,2174
1,50,Self-emp-not-inc,Exec-managerial,1,0,0,13,United-States,0,1,1,9,5,0,9,0
2,38,Private,Handlers-cleaners,1,0,0,40,United-States,0,1,0,6,1,1,0,0
3,53,Private,Handlers-cleaners,1,0,0,40,United-States,0,1,1,4,1,0,4,0
4,28,Private,Prof-specialty,0,0,0,40,Cuba,0,0,1,9,4,0,9,0
5,37,Private,Exec-managerial,0,0,0,40,United-States,0,1,1,10,5,0,10,0
6,49,Private,Other-service,0,0,0,16,Jamaica,0,0,0,4,1,0,0,0
7,52,Self-emp-not-inc,Exec-managerial,1,0,0,45,United-States,1,1,1,6,5,0,6,0
8,31,Private,Prof-specialty,0,14084,0,50,United-States,1,1,0,10,4,0,0,14084
9,42,Private,Exec-managerial,1,5178,0,40,United-States,1,1,1,9,5,0,9,5178


In [683]:
new_train.to_csv('./data/train_clean.csv', index=False)

In [101]:
## MAKE CHANGES TO TEST

In [102]:
test.head()

,is_married,job_scale,education_scale,age,sex,hours-per-week,unemployed,gov_job,self_employed,capital-gain,capital-loss,wage
0,0,2,9,39,1,40,0,1,0,2174,0,0
1,1,5,9,50,1,13,0,0,0,0,0,0
2,0,1,6,38,1,40,0,0,0,0,0,0
3,1,1,4,53,1,40,0,0,0,0,0,0
4,1,4,9,28,0,40,0,0,0,0,0,0


In [103]:
test = pd.read_csv('./data/test_data.csv', index=False)

TypeError: parser_f() got an unexpected keyword argument 'index'

In [670]:
# SEX
# Fix 'sex' column - Train (1 if male, 0 if female)
test['sex'] = test['sex'].map(lambda x: x.lower().strip())
test['sex'] = test['sex'].map(lambda x: 1 if  x == 'male' else 0)

In [104]:
# CAPITAL-NET
test['capital-net'] = test['capital-gain'] - test['capital-loss']

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [673]:
# IS_USA
test['is_USA'] = test['native-country'].map(lambda x: 1 if x.strip() == 'United-States' else 0)

In [674]:
# IS MARRIED
test['is_married'] = test['marital-status'].map(lambda x: 1 if x.strip() in ['Married-civ-spouse', 'Married-AF-spouse'] else 0)

In [675]:
# EDUCATION SCALE
test['education_scale'] = test['education'].map(lambda x: x.strip())
test['education_scale'] = test['education_scale'].map(education_map)

In [676]:
# UNEMPLOYED
test['unemployed'] = test['workclass'].map(lambda x: 1 if x.strip() in ['Never-worked', 'Without-pay', '?'] else 0)

In [677]:
# JOB SCALE
test['job_scale'] = test['occupation'].map(job_map)

In [678]:
# INTERACTION TERM
test['edu_scale * is_married'] = test['education_scale'] * test['is_married']

In [679]:
test.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country,capital-net,is_USA,is_married,education_scale,unemployed,job_scale,edu_scale * is_married
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,1,0,0,40,United-States,0,1,0,4,0,2,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,1,0,0,50,United-States,0,1,1,6,0,2,6
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,1,0,0,40,United-States,0,1,1,8,0,4,8
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,1,7688,0,40,United-States,7688,1,1,7,0,2,7
4,18,?,103497,Some-college,10,Never-married,?,Own-child,0,0,0,30,United-States,0,1,0,7,1,1,0


In [681]:
test.to_csv('./data/test_clean.csv', index=False)